# Dueling DQN

Dueling Deep Q-Networks takes a new approach to estimating Q values. One can decompose the Q value formula into two parts:
1. The advantage function - The advantage of the taking an action in that state compared to other actions : $A(s,a) = Q(s,a) - V(s)$
2. The state value function - The value of the state i.e the expected cumulative reward to be recieved following a given policy : $V(s) = \sum_{a \in \mathcal{A}}Q(s,a)$

From the advantage function we can see that $Q(s,a) = A(s,a)+V(s)$ so what dueling DQN does, is separate the estimator into two streams - one being the Advantage function and the other being the state-value function and then combine these through an special aggregation layer to get the estimate of the Q function. 

![Architecture](https://paperswithcode.com/media/methods/Screen_Shot_2020-06-03_at_3.24.01_PM.png)

The reason for this decoupling of estimators is so that the DQN can learn separately which states are or are not valuable without the need to learn the effect of each action at each state. The authors stated that there is no point in learning/calculating action values if the entire state value is bad and can be avoided e.g why calculate all actions at one state when all these actions lead to death?

This decoupling is also useful in states where actions have no impact on the environment in any important or relevant way. In this situation there is no need to calculate the value for each action.

![Example](https://miro.medium.com/max/928/1*TJrgh57v6919Ck5ZGSWZ4g.png)

The figure above is an example of the benefit - the advantage stream only learns to pay attention when cars are immediately in front of it so that it can avoid collisions but otherwise it is unnecessary to learn as actions have no relevant consequence on the environment.

The aggregation layer is important as it is the Q value is ultimately calculated - one might think it is as simple as $Q(s,a) = A(s,a)+V(s)$ but the issue with this is one of identifiability. Given a Q value we can't find A and V which is a problem for backpropagation. 

The authors propose forcing the advantage function to have zero advantage at the chosen action by using the following formula:
$Q(s,a) = V(s)+(A(s,a)-\max_{a` \in |\mathcal{A}|}A(s,a`))$

An alternative method that replaces the max operator with an average:
$Q(s,a) = V(s)+(A(s,a)-\frac{1}{|\mathcal{A}|}\sum_{a`}A(s,a`))$

This replacement loses the original semantics of V and A since it puts them off target by a constant - but this increases the stability of the optimization since advantages only need to change as fast as the mean instead of having to compensate any change to the optimal actions advantage. This is ultimately what the authors used in their paper so this implementation will use this too.

## Dueling DQN Network

The Dueling DQN takes in the state input and outputs Q values for all possible actions using the two streams and aggregator layer. There is a common feature layer and then two separate heads - This architecture might be a little deep which can slow learning down for simple problems - try removing a hidden layer from the stream heads and see if it can increase learning speed. e.g remove the first linear layer in each of the heads.

In [ ]:
from torch import nn
import torch.nn as nn
import torch.nn.functional as F
import torch

class Dueling_DQN(nn.Module):

    def __init__(self, input_size, hidden_size, nb_actions) -> None:
        super(Dueling_DQN, self).__init__()
        self.feature_layer = nn.Sequential(nn.Linear(input_size, hidden_size),
                                        nn.LeakyReLU(),
                                        nn.Linear(hidden_size, hidden_size),
                                        nn.LeakyReLU())

        self.value_function = nn.Sequential(
                                        nn.Linear(hidden_size, hidden_size),
                                        nn.LeakyReLU(),
                                        nn.Linear(hidden_size, 1),
                                        nn.LeakyReLU())

        self.advantage_function = nn.Sequential(
                                        nn.Linear(hidden_size, hidden_size),
                                        nn.LeakyReLU(),
                                        nn.Linear(hidden_size, nb_actions),
                                        nn.LeakyReLU())
        
    def forward(self,obs):
        feature_state = self.feature_layer(obs)
        state_value = self.value_function(feature_state)
        advantage_values = self.advantage_function(feature_state)
        q_values = state_value + (advantage_values - advantage_values.mean())
        
        return q_values
        

## Replay Memory

DQNs make use of experience replay memory - this stores previous transitions to allow for data efficient and non correlated learning to take place.

In [ ]:
from collections import namedtuple, deque
import random
import numpy as np

Transition = namedtuple(
    "Transition", ("state", "action", "reward", "next_state", "done"))

class Replay_Memory:

    def __init__(self, size):
        self.memory = deque([], maxlen=size)

    def push(self, *args):
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        if batch_size > len(self.memory):
            return None
        return random.sample(self.memory, batch_size)


## Agent 
The following is the agent class:

In [ ]:
from torch.distributions import Categorical, Normal
from torch.optim import AdamW
import torch

class Dueling_DQN_Agent:

    def __init__(self,input_size,hidden_size,nb_actions,discount_factor=0.9,learning_rate=0.0001,epsilon_start=0.9,epsilon_end=0.05,epsilon_anneal_over_steps=10000,batch_size=256,update_frequency = 1000,replay_memory_size=100000):
        
        # Create the online network that the agent uses to select actions
        self.online_net = Dueling_DQN(input_size,hidden_size,nb_actions)

        # Create the target network that the agent uses in it's updates
        self.target_net = Dueling_DQN(input_size,hidden_size,nb_actions)

        self.num_actions = nb_actions

        # Set the target net's initial weights to equal the online net
        self.target_net.load_state_dict(self.online_net.state_dict())

        self.gamma = discount_factor
        
        # User Huber Loss function - others can be use such as MSE
        self.loss_function = torch.nn.SmoothL1Loss()

        # DQN optimizer
        self.optimizer = AdamW(self.online_net.parameters(),learning_rate)
        
        # Set initial and final epsilon values for epsilon greedy and choose the number of environment steps to anneal over
        self.epsilon_end = epsilon_end
        self.epsilon_start = epsilon_start
        self.epsilon_anneal_over_steps = epsilon_anneal_over_steps
        self.step_no = 0

        # Create Replay Memory and assign batch_size
        self.replay = Replay_Memory(replay_memory_size)
        self.batch_size = batch_size
        
        # Set update Frequency
        self.update_frequency = update_frequency
        
        # Best Weights for Eval
        self.best_weights = self.online_net.state_dict()

    def get_epsilon(self):
        """
        Get current epsilon value according to agents total step number in the environment
        """
        eps = self.epsilon_end
        if self.step_no < self.epsilon_anneal_over_steps:
            eps = self.epsilon_start - self.step_no * \
                ((self.epsilon_start - self.epsilon_end) /
                 self.epsilon_anneal_over_steps)
        return eps
        
    def act(self,obs):
        # Increment global step count
        self.step_no+=1

        if np.random.uniform() > self.get_epsilon():
            # Dont store gradients when acting in the environment
            with torch.no_grad():
                return torch.argmax(self.online_net(obs),dim=-1).view(1)
        else:
            return torch.tensor([random.randrange(self.num_actions)], dtype=torch.long)

    # store experience in replay memory
    def cache(self, state, action, reward, next_state, done):
        self.replay.push(state, action, reward, next_state, done)

    def train(self):
        self.online_net.train()
        self.target_net.train()
    
    def eval(self):
        self.online_net.load_state_dict(self.best_weights)
        self.online_net.eval()
        self.target_net.eval()

    def update_model(self):

        # Get minibatch of data from experience buffer
        batch = self.replay.sample(self.batch_size)

        # If memory doesnt have enough transitions
        if batch == None:
            return

        # Format batch to get a tensor of states, actions, rewards, next states and done booleans
        batch_tuple = Transition(*zip(*batch))
        state = torch.stack(batch_tuple.state)
        action = torch.stack(batch_tuple.action)
        reward = torch.stack(batch_tuple.reward)
        next_state = torch.stack(batch_tuple.next_state)
        done = torch.stack(batch_tuple.done)

        self.optimizer.zero_grad()

        # Get the Q values of the online nets current state and actions
        Q_Values = self.online_net(state).gather(1, action).squeeze()

        # Get the max Q values of the target nets next state
        Q_Targets = reward + (1 - done.float()) * self.gamma * self.target_net(next_state).max(dim=-1)[0].detach()
        
        # Calculate loss
        loss = self.loss_function(Q_Values, Q_Targets)

        # Calculate the gradients
        loss.backward()

        # Perform optimization step
        self.optimizer.step()

        return loss.item()
    

    def update_target(self):
        """
        Update the target nets weights
        """
        self.target_net.load_state_dict(self.online_net.state_dict())

    def update_best_weights(self):
        self.best_weights = self.online_net.state_dict()


## Training Loop
The last thing needed is the training loop.

In [ ]:
from collections import deque
import gym
from numpy import double
import torch
import argparse

def train(agent, env, num_episodes, num_steps, log_every=20, render_from=1000):
    """
    :param agent: the agent to be trained.
    :param env: the gym environment.
    :param num_episodes: the number of episodes to train.
    :param num_steps: the max number of steps possible per episode.
    :param log_every: The frequency of logging. Default logs every 20 episodes.
    :param render_from: The episode number to start rendering to screen to allow one to view agent. Rendering significantly slows down training.
    """
    
    # Running Average Reward Memory
    running_avg_reward = deque(maxlen=100)
    best_running_avg = 0
    agent.train()
    for episode in range(1,num_episodes+1):
        # Starting state observation
        obs = torch.tensor(env.reset(),dtype=torch.float32)
        reward_total = 0
        for step in range(num_steps):
            if episode>render_from:
                env.render()

            # return chosen action
            action = agent.act(obs)
            
            # Take a step in the environment with the action drawn
            next_obs , reward, done, info = env.step(action.item())
            
            # Just for logging
            reward_total+=reward

            # change next state into tensor for update
            next_obs = torch.tensor(next_obs,dtype=torch.float32)
            
            # change reward into tensor for update
            reward = torch.tensor(
                reward, dtype=torch.float32)

            # Store transition in replay memory
            agent.cache(obs,action,reward,next_obs,torch.tensor(done))

            # Perform update
            agent.update_model()
            
            # If the number of steps has elapsed then perform update
            if agent.step_no % agent.update_frequency == 0:
                agent.update_target()

            # Set the current state to the next state
            obs = next_obs
            
            # if done then log and break
            if done:
                if episode % log_every ==0 and len(running_avg_reward)>0:
                    running_avg = sum(running_avg_reward)/len(running_avg_reward)
                    if running_avg > best_running_avg:
                        best_running_avg = running_avg
                        agent.update_best_weights()
                        
                    print("Episode {0:4d} finished after {1:4d} timesteps with a total reward of {2:3.1f} | Running Average: {3:3.2f}".format(episode,step+1,reward_total,running_avg))
                break
        running_avg_reward.append(reward_total)
        

    env.close()

In [ ]:

def eval(agent, env, num_episodes, num_steps, log_every=20):
    """
    :param agent: the agent to be trained.
    :param env: the gym environment.
    :param num_episodes: the number of episodes to eval.
    :param num_steps: the max number of steps possible per episode.
    :param log_every: The frequency of logging. Default logs every 20 episodes.
    """
    agent.eval()
    # Running Average Reward Memory
    running_avg_reward = deque(maxlen=100)
    for episode in range(1,num_episodes+1):
        # Starting state observation
        obs = torch.tensor(env.reset(),dtype=torch.float32)
        reward_total = 0
        for step in range(num_steps):
           
            env.render()
            # return chosen action
            action = agent.act(obs)
            
            # Take a step in the environment with the action drawn
            next_obs , reward, done, info = env.step(action.item())
            
            # Just for logging
            reward_total+=reward

            # change next state into tensor for update
            next_obs = torch.tensor(next_obs,dtype=torch.float32)
            
            # Set the current state to the next state
            obs = next_obs
           
            # if done then log and break
            if done:
                if episode % log_every ==0 and len(running_avg_reward)>0:
                    running_avg = sum(running_avg_reward)/len(running_avg_reward)
                    print("Episode {0:4d} finished after {1:4d} timesteps with a total reward of {2:3.1f} | Running Average: {3:3.2f}".format(episode,step+1,reward_total,running_avg))
                break
        running_avg_reward.append(reward_total)
    env.close()

## Time to Train

In [ ]:
env = gym.make("CartPole-v0")


nb_actions = env.action_space.n

# Hyper parameters
HIDDEN_SIZE = 512
GAMMA = 0.99
LEARNING_RATE = 0.0005
EPISODES_TO_TRAIN = 700
EPISODES_TO_EVAL = 20
MAX_STEPS = 201
LOG_EVERY = 20
RENDER_FROM_EP = 2000
EPSILON_START=0.9
EPSILON_END=0.01
EPSILON_ANNEAL_OVER_STEPS=10000
BATCH_SIZE=512
UPDATE_FREQUENCY = 1500
REPLAY_MEMORY_SIZE=500000


agent = Dueling_DQN_Agent(env.observation_space.shape[0],HIDDEN_SIZE,nb_actions,GAMMA,LEARNING_RATE,EPSILON_START,EPSILON_END,EPSILON_ANNEAL_OVER_STEPS,BATCH_SIZE,UPDATE_FREQUENCY,REPLAY_MEMORY_SIZE)

print("Training...")

train(agent,env,EPISODES_TO_TRAIN,MAX_STEPS,LOG_EVERY,RENDER_FROM_EP)

print("Evaluating...")

eval(agent,env,EPISODES_TO_EVAL,MAX_STEPS,log_every=1)